In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gzip
import json
import ast
import pandas as pd


#### 1. Carga de Datos desde los achivos json

In [3]:
data= []
file = gzip.open('data/steam_games.json.gz', 'r')
for linea in file:
    data.append(json.loads(linea))
df_steam_games= pd.DataFrame(data)


In [4]:
data= []
file = gzip.open('data/user_reviews.json.gz', 'r')
for linea in file:
    data.append(ast.literal_eval(linea.decode('utf-8')))
df_user_reviews= pd.DataFrame(data)

In [3]:
data= []
file = gzip.open('data/users_items.json.gz', 'r')
for linea in file:    
    data.append(ast.literal_eval(linea.decode('utf-8')))
df_user_items= pd.DataFrame(data)

#### 2. Borrar datos nulos y columas inncesarias de los DataSets. Tranformar columnas Anidadas

- **DataFrame: "df_steam_games"**

In [6]:
df_steam_games = df_steam_games.explode('genres') # se crean filas con los vaores aninados de genres

In [7]:
df_steam_games['release_date']= pd.to_datetime(df_steam_games['release_date'], errors='coerce')
df_steam_games['release_year']= df_steam_games['release_date'].dt.year
df_steam_games['release_year'].fillna(0, inplace = True)
df_steam_games['release_year']= df_steam_games['release_year'].astype(int)

C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_18264\3836482271.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_steam_games['release_year'].fillna(0, inplace = True)


In [8]:
df_steam_games= df_steam_games.dropna(how='all') # Elimina todas las finas que continen todos los valores nulos
df_steam_games= df_steam_games.dropna(subset=['app_name']) # Elimina filas con app_name nulo
df_steam_games= df_steam_games.dropna(subset=['id']) # Elimina filas con id nulos
df_steam_games= df_steam_games.drop(['title','release_date','url','reviews_url','tags', 'specs', 'early_access', 'publisher', 'developer', 'price'], axis=1) # Eliminar columnas innecesarias
df_steam_games= df_steam_games.rename(columns={'app_name': 'game_name'}) # cambiar normbre de la columna
df_steam_games= df_steam_games.rename(columns={'id': 'game_id'}) # cambiar normbre de la columna

In [9]:
df_steam_games.isna().sum()

genres          3282
game_name          0
game_id            0
release_year       0
dtype: int64

In [10]:
df_steam_games.head(5)

,genres,game_name,game_id,release_year
88310,Action,Lost Summoner Kitty,761140,2018
88310,Casual,Lost Summoner Kitty,761140,2018
88310,Indie,Lost Summoner Kitty,761140,2018
88310,Simulation,Lost Summoner Kitty,761140,2018
88310,Strategy,Lost Summoner Kitty,761140,2018


- **DataFrame: "df_user_reviews"**

In [11]:
lista_datos = []
id_usuarios = []
# Iterar sobre los registros del DataFrame
for index, fila in df_user_reviews.iterrows():    
    for elemento in fila['reviews']:
        id_usuarios.append(fila['user_id'])
        elem_review = ast.literal_eval(str(elemento))
        lista_datos.append(elem_review)

In [12]:
df_user_reviews = pd.DataFrame(lista_datos)
df_user_reviews['user_id']= id_usuarios

In [13]:
#Funcion para calcular el analisis de sentimientos

analizador = SentimentIntensityAnalyzer() 
def analisis_sentimiento(review):    
    puntuacion = analizador.polarity_scores(review)
    if puntuacion['compound'] > 0.5:
        return 2
    if (puntuacion['compound'] < 0.5) & (puntuacion['compound'] >= 0):
        return 1
    else:
        return 0

In [14]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(analisis_sentimiento) # Se crea un columna con el analisis de sentimmientos calculados

In [15]:
df_user_reviews['sentiment_analysis'].value_counts() # Contar los valores de analisis de sentimientos

sentiment_analysis
2    31581
1    19201
0     8523
Name: count, dtype: int64

In [16]:
df_user_reviews['date'] = pd.to_datetime((df_user_reviews['posted']), format='Posted %B %d, %Y.', errors='coerce') # Crear una fila fecha con tranformacion de posted un fomrato fecha
df_user_reviews['year'] = df_user_reviews['date'].dt.year
df_user_reviews['year'].fillna(0, inplace = True)
df_user_reviews['year'] = df_user_reviews['year'].astype(int)

C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_18264\756992562.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_user_reviews['year'].fillna(0, inplace = True)


In [17]:
df_user_reviews = df_user_reviews.rename(columns={'item_id': 'game_id'}) # cambiar normbre de la columna
df_user_reviews = df_user_reviews[['user_id','year', 'game_id', 'recommend', 'sentiment_analysis']] # Se seleccionan solo las culumnas necesarias
df_user_reviews.head()

,user_id,year,game_id,recommend,sentiment_analysis
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2
3,js41637,2014,251610,True,2
4,js41637,2013,227300,True,2


In [18]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   year                59305 non-null  int32 
 2   game_id             59305 non-null  object
 3   recommend           59305 non-null  bool  
 4   sentiment_analysis  59305 non-null  int64 
dtypes: bool(1), int32(1), int64(1), object(2)
memory usage: 1.6+ MB


- **DataFrame: "df_user_items"**

In [4]:
lista_datos = []
id_usuarios = []
# Iterar sobre los registros del DataFrame
for index, fila in df_user_items.iterrows(): 
    for elemento in fila['items']:
        id_usuarios.append(fila['user_id'])
        item = ast.literal_eval(str(elemento))
        lista_datos.append(item)

In [5]:
df_user_items = pd.DataFrame(lista_datos)   # se sobre escribe el dataFrame nuevmamentes con la lista de datos anidadas
df_user_items['user_id']= id_usuarios       # Se agrega la columna user_id

In [6]:
df_user_items= df_user_items.rename(columns={'item_name': 'game_name'}) # cambiar normbre de la columna
df_user_items= df_user_items.rename(columns={'item_id': 'game_id'}) # cambiar normbre de la columna
df_user_items['game_id']= df_user_items['game_id'].astype(int)
df_user_items = df_user_items[['user_id','game_id', 'game_name', 'playtime_forever']] ## Se seleccionan solo las culumnas necesarias

In [7]:
df_user_items.drop_duplicates(inplace=True)     # Elimina filas duplicadas

In [8]:
df_user_items.head()

,user_id,game_id,game_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [9]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094092 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   game_id           int32 
 2   game_name         object
 3   playtime_forever  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 174.9+ MB


#### 3. Exportar los Dataframes transformados a archivos CSV y parquet

In [25]:
df_steam_games.to_csv('DataSet_tranformados/games.csv', index=False)    #Exportar df_steam_games a un archivo games.cvs

In [26]:
df_user_reviews.to_csv('DataSet_tranformados/reviews.csv', index=False) #Exportar df_user_reviews a un archivo reviews.cvs

In [10]:
df_user_items.to_parquet('DataSet_tranformados/items.parquet', index=False) #Exportar df_user_reviews a un archivo items.parquet

In [11]:
#df_user_items.iloc[1,-1]
#user_reviews_temp = user_reviews.explode('reviews')
#user_reviews_temp.head(10)

In [65]:
#frase= (df_user_reviews.iloc[132,-1])
#print(frase)
#print(analisis_sentimiento(frase))
#print(analisis_sentimiento("Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!"))